In [1]:
import numpy as np
import cv2

In [2]:
# Örnek 5.3: Lucas - Kanade ile Optik Akış

cornerDetectionCriteria = dict(maxCorners = 10, qualityLevel = 0.3, minDistance = 7, blockSize = 7)
lucasCriteria = dict(winSize=(200,200),maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_COUNT,10,0.03))

video = cv2.VideoCapture(0)

ret, prevFrame = video.read()
grayPrevFrame = cv2.cvtColor(prevFrame, cv2.COLOR_BGR2GRAY)

prevPoints = cv2.goodFeaturesToTrack(grayPrevFrame,mask=None,**cornerDetectionCriteria)
mask = np.zeros_like(prevFrame)

if video.isOpened() == False:
    print("Doğru gitmeyen bir şeyler var.")

while video.isOpened:
    ret, currentFrame = video.read()
    grayCurrentFrame = cv2.cvtColor(currentFrame, cv2.COLOR_BGR2GRAY)
    
    nextPoints, status, errors = cv2.calcOpticalFlowPyrLK(grayPrevFrame,grayCurrentFrame,prevPoints,None,**lucasCriteria)
    
    newMotion = nextPoints[status==1]
    oldMotion = prevPoints[status==1]
    
    for i, (new, prev) in enumerate(zip(newMotion,oldMotion)):
        x_newPos, y_newPos = new.ravel()
        x_prevPos, y_prevPos = prev.ravel()
        
        mask = cv2.line(mask,(x_newPos,y_newPos),(x_prevPos,y_prevPos),(0,255,0),2)
        currentFrame = cv2.circle(currentFrame,(x_newPos,y_newPos),8,(0,0,255),-1)
    
    img = cv2.add(currentFrame,mask)
    cv2.imshow('takip',img)
    
    k = cv2.waitKey(10) & 0xFF
    if k ==27:
        break

    grayPrevFrame = grayCurrentFrame.copy()
    nextPoints = newMotion.reshape(-1,1,2)
    
cv2.destroyAllWindows()
video.release()

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
